In [13]:
import os
import sys
from pathlib import Path

import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torchvision import datasets, transforms
from torchvision.utils import save_image

# from gendis.config import config
from gendis.datasets import MultiDistrDataModule
from gendis.variational.conv_vae import Conv_VAE

sys.path.append("../vae_reduction/")
# from config import config

In [9]:
def make_model(config):
    model_type = config.model_type
    model_config = config.model_config

    if model_type not in vae_models.keys():
        raise NotImplementedError("Model Architecture not implemented")
    else:
        return vae_models[model_type](**model_config.dict())


def load_vae_model(config, data_root, model_root):
    config.model_config.data_path = data_root
    model = make_model(config)
    checkpoint = torch.load(model_root, map_location=torch.device("cpu"))
    model.load_state_dict(checkpoint["state_dict"])
    return model


def make_vae_reduction_dataset(model, data_root, new_root):
    """
    :param model: vae model
    :param data_root: old data root
    :param new_data_root: new data root. please notice your data file will have the same file name as the original data,
            so please find new data under the new_data_root folder. it should be under new_data_root/CausalBarMNIST/chain/
    """

    assert data_root != new_root

    # please do not change this transform sincethe vae model used this data_transform to train.
    # This is only for vae model.
    data_transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Lambda(lambda x: 2 * x - 1.0)]
    )

    # please do not set
    data_module = MultiDistrDataModule(
        root=data_root,
        graph_type="chain",
        batch_size=100,
        stratify_distrs=False,
        transform=data_transform,
        num_workers=1,
        train_size=1.0,
    )

    data_module.setup()

    all_x = []
    all_labels = []
    all_targets = []

    for images, labels, targets in data_module.train_dataloader():
        with torch.no_grad():
            x, _ = model.encode(images)

        all_x.extend(x)
        all_labels.extend(labels)
        all_targets.extend(targets)

    if not os.path.exists(new_root):
        os.makedirs(new_root)
    if not os.path.exists(new_root + "CausalBarMNIST/"):
        os.makedirs(new_root + "CausalBarMNIST/")
    if not os.path.exists(new_root + "CausalBarMNIST/" + "chain/"):
        os.makedirs(new_root + "CausalBarMNIST/" + "chain/")

    imgs_fname = new_root + "CausalBarMNIST/" + "chain/" + "chain-imgs-train.pt"
    labels_fname = new_root + "CausalBarMNIST/" + "chain/" + "chain-labels-train.pt"
    targets_fname = new_root + "CausalBarMNIST/" + "chain/" + "chain-targets-train.pt"

    torch.save(all_x, imgs_fname)
    torch.save(all_labels, labels_fname)
    torch.save(all_targets, targets_fname)

In [16]:
root = Path("/Users/adam2392/pytorch_data/data")
model_root = root / "vae-reduction" / "conv-vae_alpha_1024_dim_32.ckpt"

data_root = Path("/Users/adam2392/pytorch_data/CausalBarMNIST/")

# load vae model
# model = load_vae_model(data_root, model_root)
model = Conv_VAE.load_from_checkpoint(model_root)
model.eval()
# set new data root, must be different with the data_root
new_root = data_root / "causalbar_reduction_dat"

# make new data
make_vae_reduction_dataset(model, data_root, new_root)

/Users/adam2392/miniforge3/envs/cdrl/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/migration.py:195: PossibleUserWarning: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
  rank_zero_warn(
Lightning automatically upgraded your loaded checkpoint from v1.3.8 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../pytorch_data/data/vae-reduction/conv-vae_alpha_1024_dim_32.ckpt`


FileNotFoundError: [Errno 2] No such file or directory: '../dat/CausalBarMNIST/chain/chain-imgs-train.pt'

In [ ]:
# demo to load dataloader. please make sure transform is None. d
new_data_module = MultiDistrDataModule(
    root=new_root,
    graph_type="chain",
    batch_size=32,
    stratify_distrs=True,
    transform=None,
    num_workers=1,
)
new_data_module.setup()

dataiter = iter(new_data_module.train_dataloader())

x, labels, targets = next(dataiter)
print(x.shape, labels.shape, targets.shape)
with torch.no_grad():
    images = model.decoder(x)
images = model.scale_image(images)
save_image(
    images,
    "samples.png",
)

print(labels)